# Connection to Postgres

In [11]:
import { load } from '@std/dotenv'

// Ensure env vars are loaded before creating config
await load({ export: true, envPath: './temp/.env.prod' })

const DB_CONFIG = {
  hostname: Deno.env.get('PGHOST') ?? '127.0.0.1',
  port: Number(Deno.env.get('PGPORT') ?? 5432),
  user: Deno.env.get('PGUSER') ?? 'deno',
  password: Deno.env.get('PGPASSWORD') ?? '',
  database: Deno.env.get('PGDATABASE') ?? 'denodb',
}

console.log('Database configuration loaded')

Database configuration loaded


In [12]:
import { Client } from 'postgres'

let client: Client | null = null

async function getClient(): Promise<Client> {
  if (!client) {
    client = new Client(DB_CONFIG)
    await client.connect()
  }
  return client
}

await getClient().then(() => {
  console.log('Connected to the database')
})
.catch((error) => {
  console.error('Failed to connect to the database:', error)
})

async function query<T = unknown>(
  sql: string,
  params: unknown[] = [],
): Promise<T[]> {
  try {
    const dbClient = await getClient()
    const result = await dbClient.queryObject<T>(sql, params)
    return result.rows
  } catch (error) {
    console.error('Database query error:', error)
    throw error
  }
}



TLS connection failed with message: invalid peer certificate: Other(OtherError(CaUsedAsEndEntity))
Defaulting to non-encrypted connection


Connected to the database


Get All the todos

In [13]:
interface Todo {
  id: number
  title: string
  completed: boolean
  created_at: Date
  updated_at: Date
}

await query<Todo>('SELECT * FROM todos ORDER BY created_at DESC').then((todos) => {
  console.log('Todos:', todos)
}).catch((error) => {
  console.error('Error fetching todos:', error)
})

Todos: [
  {
    id: 1,
    title: "Learn Deno",
    completed: false,
    created_at: 2025-08-30T03:32:05.642Z,
    updated_at: 2025-08-30T03:33:00.811Z
  },
  {
    id: 2,
    title: "Build HTMX app",
    completed: false,
    created_at: 2025-08-30T03:32:05.642Z,
    updated_at: 2025-08-30T03:33:02.820Z
  },
  {
    id: 3,
    title: "Deploy to production",
    completed: true,
    created_at: 2025-08-30T03:32:05.642Z,
    updated_at: 2025-08-30T03:38:16.724Z
  }
]
